In [1]:
%%time
from algoutils import *

raw_courses = read('data/courses.txt')
raw_industries = read('data/industries.txt')
raw_job_industries = read('data/job_industries.txt')

Wall time: 5 ms


In [2]:
%%time
# note: enchant module and library should be installed
from cleaning import clean

clean_courses = clean(raw_courses, 'course', 'data/course_dictionary.txt')
clean_industries = clean(raw_industries, 'industry', 'data/industry_dictionary.txt')
clean_job_industries = clean(raw_job_industries, 'industry', 'data/industry_dictionary.txt')

Wall time: 12.6 s


In [3]:
%%time
# note: nltk module and nltk.wordnet corpus should be installed
from clustering import stem_cluster
 
course_clusters = stem_cluster(clean_courses)
industry_clusters = stem_cluster(clean_industries)
job_industry_clusters = stem_cluster(clean_job_industries, mode=5)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Wall time: 4.13 s


In [4]:
sorted(course_clusters)[:10]

['ACCOUNTANCY',
 'ACCOUNTING',
 'ADMINISTRATION',
 'ADVERTISING',
 'AGRIBUSINESS',
 'AGRICULTURAL',
 'AGRICULTURE',
 'ANIMATION',
 'APPLIED',
 'ARCHITECTURAL']

In [5]:
sorted(industry_clusters)[:10]

['ACCOUNTING',
 'ADVERTISING',
 'AGENCY',
 'ASSISTANT',
 'AUTOMOTIVE',
 'BANKING',
 'BEAUTY',
 'BPO',
 'BROKERAGE',
 'BUILDING']

In [6]:
sorted(job_industry_clusters)[:10]

['BUSINESS',
 'CALLCENTER',
 'COMPUTER',
 'CONSTRUCTION',
 'DEPARTMENT',
 'DESIGN',
 'DEVELOPMENT',
 'DISTRIBUTION',
 'EDUCATION',
 'ENGINE']

In [8]:
%%time

from analysis import *

# note: pandas and sklearn modules should be installed
# initializes a likelihood data frame based on row and column labels.
# Performs a content-based filtering technique using features.
# vectorizer can be 'tfidf' or 'count'

CourseIndustry = likelihood_matrix(course_clusters, industry_clusters, vectorizer='tfidf', ngram_range=(3,4))

Wall time: 342 ms


In [9]:
CourseIndustry.dataframe.head()

,ENGINE,REPAIR,HARDWARE,PRODUCT,PARTS,CALLCENTER,LINE,CAFE,CUSTOMER,OFFICE,...,STORE,BUILDING,CITY,PLANT,PUBLISHING,FABRICATION,HEALTH,ASSISTANT,DISTRIBUTOR,BROKERAGE
LINGUISTICS,0,0,0,0,0,0.000000,0.111031,0,0,0,...,0,0.023,0,0,0.020312,0.000000,0,0.045016,0.042213,0
HARDWARE,0,0,1,0,0,0.000000,0.000000,0,0,0,...,0,0.000,0,0,0.000000,0.000000,0,0.000000,0.000000,0
AGRICULTURAL,0,0,0,0,0,0.000000,0.000000,0,0,0,...,0,0.000,0,0,0.000000,0.052721,0,0.000000,0.000000,0
ELEMENTARY,0,0,0,0,0,0.034037,0.000000,0,0,0,...,0,0.000,0,0,0.000000,0.000000,0,0.000000,0.000000,0
NCII,0,0,0,0,0,0.000000,0.000000,0,0,0,...,0,0.000,0,0,0.000000,0.000000,0,0.000000,0.000000,0


In [10]:
%%time

# note: pandas and sklearn modules should be installed
# derives the change in likelihood if an entry

# computes delta likelihood using the following steps: (non-verbatim)
# clabels = df.column_labels
# rlabels = df.row_labels
# vr = [cos_similarity(ur, label) for label in rlabels]
# vc = [cos_similarity(vr, label) for label in clabels]
# dL = vr * vc^T

d1 = CourseIndustry.delta('CALLCENTE', 'BUSINESS')
d2 = CourseIndustry.delta('MARKETING', 'RESEARCH & DEVELOPMENT')
d3 = CourseIndustry.delta('AGRICULTURE', 'FINANCE')

Wall time: 30 ms


In [11]:
%%time

# dynamic update of likelihood matrix
# just accumulate small likelihoods
# accounts for relative likelihood and not absolute percentage

users = [
    ('COMPUTER SCIENCE', 'INFORMATION TECHNOLOGY'),
    ('ACCOUNTING', 'BUSINESS'),
    ('BUSINESS ADMINSTRATION', 'FINANCE'),
    ('MARKETING', 'FINANCE'),
    ('MATHEMATICS', 'FINANCE'),
    ('SOCIAL SCIENCE', 'CONSULATION'),
    ('LITERATURE', 'WRITING'),
    ('MANAGEMENT', 'BANKING'),
    ('SCIENCES', 'RESEARCH')
]

for course, industry in users:
    CourseIndustry.add_match(course, industry)

Wall time: 64 ms


In [12]:
# get specific likelihood coefficient of a pair of clusters
# e.g. likelihood of courses from the BUSINESS industry

# note: 1.00 does not mean 100%
# likelihood metric is relative
# the more data, the more accurate the prediction

%time CourseIndustry.dataframe.BUSINESS.nlargest(10)

Wall time: 1e+03 µs


BUSINESS        1.000000
ACCOUNTING      1.000000
AGRIBUSINESS    0.751331
ACCOUNTANCY     0.592993
WRITING         0.164020
DRAFTING        0.139844
MARKETING       0.127857
NURSING         0.110499
ADVERTISING     0.076887
MARINE          0.072553
Name: BUSINESS, dtype: float64

In [13]:
%%time

# transpose data frame for rows
CourseIndustry.dataframe.T.SCIENCE.nlargest(6)

Wall time: 1 ms


RESEARCH       0.952430
INFORMATION    0.632183
TECHNOLOGY     0.462987
CONSULTING     0.412805
CONSULTANCY    0.370199
EDUCATION      0.258203
Name: SCIENCE, dtype: float64

In [14]:
%%time

# add count from a specific matching pair
# note: this is synonymous to CourseIndustry.dataframe += CourseIndustry.delta('PHYSICS', 'REPAIR SUPPLY')

CourseIndustry.add_match('PHYSICS', 'RESEARCH')

Wall time: 4 ms


In [15]:
%%time

# returns a vector of recommend likelihood of a course from a specific industry
# since courses are rows, we find column matches
# we can also make the method output percentages

matches = CourseIndustry.find_column_matches('PHYSICS', with_labels = True, percentage = True)

Wall time: 1e+03 µs


In [16]:
# rank recommendations with the highest percentage
matches.nlargest(5)

RESEARCH       55.662739
MEDICAL         5.643027
ELECTRICAL      5.305848
ELECTRONICS     4.931548
FINANCE         4.829725
dtype: float64

In [17]:
%%time

# we can also get the recommendation score of a specific pair
# this is equivalent to SV1^T * LM * SV2^T
# where SV is similarity_vector, LM is likelihood matrix

CourseIndustry.recommendation_score('PHYSICS', 'SCIENTIFIC RESEARCH')

Wall time: 6 ms


55.353636624354287

In [18]:
# You can also get the dot product of data frames
# to get bridged likelihood

IndustryJob = likelihood_matrix(industry_clusters, job_industry_clusters, vectorizer='tfidf', ngram_range=(3,4))
CourseJob = CourseIndustry * IndustryJob

In [19]:
CourseJob.recommendation_score('PHYSICS', 'MANAGEMENT')

0.28752848943447878

In [20]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'ARCHITECTURE')

2.0381146684142672

In [21]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'TECHNOLOGY')

37.428856177544375

In [22]:
# after adding a match to course and a job, recommendation score should go up
CourseJob.add_match('COMPUTER SCIENCE', 'TECHNOLOGY')
CourseJob.add_match('PROGRAMMING', 'COMPUTER')

In [23]:
CourseJob.recommendation_score('PROGRAMMING', 'COMPUTER')

69.02283501675754